# 1. Data Analysis

In [28]:
using Pkg
using Plots
using Random
using GLM
using DataFrames
using LaTeXStrings
using Statistics
using StatsBase
using CSV
Pkg.add("CSV")

1.1. Import the data set. Make sure the column names are imported as intended.

In [29]:
data = CSV.read("wage2015_subsample_inference.csv", DataFrame);
first(data,5)

Row,rownames,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Int64
1,10,9.61538,2.26336,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,3600.0,11,8370.0,18
2,12,48.0769,3.8728,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,3050.0,10,5070.0,9
3,15,11.0577,2.40313,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,3.24,5.832,10.4976,6260.0,19,770.0,4
4,18,13.9423,2.63493,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,25.0,6.25,15.625,39.0625,420.0,1,6990.0,12
5,19,28.8462,3.36198,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,2015.0,6,9470.0,22


1.2. Are there missing values? Display the number of missings (if any) of each variable in the data set.

In [30]:
# Obtener el número de valores faltantes por columna usando describe
missing_counts = describe(data).nmissing
missing_df = DataFrame(Column = names(data), MissingCount = missing_counts)
println(missing_df)
# Verificar si hay o no valores faltantes
if all(count -> count == 0, missing_counts)
    println("There are no missing values")
else
    println("There are missing values")
end

1.3. Report descriptive statistics of the variables (mean, standard deviation, percentiles, etc.). Interpret your results.

In [31]:
describe(data)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,DataType
1,rownames,15636.3,10,15260.0,32643,0,Int64
2,wage,23.4104,3.02198,19.2308,528.846,0,Float64
3,lwage,2.97079,1.10591,2.95651,6.2707,0,Float64
4,sex,0.444466,0.0,0.0,1.0,0,Float64
5,shs,0.023301,0.0,0.0,1.0,0,Float64
6,hsg,0.243883,0.0,0.0,1.0,0,Float64
7,scl,0.278058,0.0,0.0,1.0,0,Float64
8,clg,0.31767,0.0,0.0,1.0,0,Float64
9,ad,0.137087,0.0,0.0,1.0,0,Float64


In [32]:
# Se crea un DataFrame con los nombres de las columnas y calcular percentiles
percentiles_df = DataFrame(Column = String[], p10 = Float64[], p25 = Float64[], p50 = Float64[], p75 = Float64[], p90 = Float64[])

# Iterando sobre cada columna numérica del DataFrame
for (col_name, col_data) in zip(names(data), eachcol(data))
    # Se comprueba si la columna es numérica
    if eltype(col_data) <: Number
        # Se calcula los percentiles específicos
        p10 = quantile(col_data, 0.10)
        p25 = quantile(col_data, 0.25)
        p50 = quantile(col_data, 0.50)
        p75 = quantile(col_data, 0.75)
        p90 = quantile(col_data, 0.90)
        
        # se agregan los resultados al DataFrame de percentiles
        push!(percentiles_df, (Column = col_name, p10 = p10, p25 = p25, p50 = p50, p75 = p75, p90 = p90))
    end
end

# Se imprimen los percentiles calculados
println(percentiles_df)

Desviación estandar

In [33]:
# DataFrame con los nombres de las columnas y la desviación estándar
std_df = DataFrame(Column = String[], StdDev = Float64[])

# Iteración sobre cada columna del DataFrame
for (col_name, col_data) in zip(names(data), eachcol(data))
    # Comprobando que la columna sea numérica
    if eltype(col_data) <: Number
        # Calculo de la desviación estándar de la columna
        std_dev = std(skipmissing(col_data))
        
        # Resultados al DataFrame
        push!(std_df, (Column = col_name, StdDev = std_dev))
    end
end
println(std_df)

1.4. How many women with a college graduate degree (clg) or above have a wage corresponding to the 25% richest of the sample? Report the dataframe corresponding to this conditions and the result.

In [34]:
# Se filtra y cuenta a las mujeres con título universitario o superior en el 25% más rico de la muestra
rich_ed_women = filter(row -> (row.sex == 1) && ((row.clg == 1) || (row.ad == 1)) && (row.wage >= quantile(skipmissing(data.wage), 0.75)), data)

# Impresión 
println("Número de mujeres educadas en el 25% más rico: ", nrow(rich_ed_women))
println("DataFrame de mujeres educadas en el 25% más rico:")
println(rich_ed_women)

1.5. How many men with a high school graduate degree (hsg) or below have a wage corresponding to the 25% richest of the sample? Report the dataframe corresponding to this conditions and the result.

In [35]:
# Se filtra y cuenta a los hombres con título de escuela secundaria o menor en el 25% más rico de la muestra
rich_hsg_men = filter(row -> (row.sex == 0) &&  # Filtra hombres
                             ((row.hsg == 1) || (row.hsg == 1)) &&  # Filtro por nivel educativo (secundaria o menor)
                             (row.wage >= quantile(skipmissing(data.wage), 0.75)), data)  # Filtro por salario en el 25% más alto
# Impresión 
println("Número de hombres con educación secundaria o menor en el 25% más rico: ", nrow(rich_hsg_men))
println("DataFrame de hombres con educación secundaria o menor en el 25% más rico:")
println(rich_hsg_men)

1.6. Create two dataframes. One containing only the log(wage) and the other containig every variable of the data set but the wage related variables.

In [36]:
# DataFrame para 'lwage' que es el log(wage)
df_lwage = data[:, [:lwage]]

# DataFrame para el resto de las columnas a excepción de 'rownames' 'wage' y 'lwage'
data2 = select(data, Not([:rownames, :wage, :lwage]))

display(df_lwage)
display(data2)

Row,lwage
,Float64
1,2.26336
2,3.8728
3,2.40313
4,2.63493
5,3.36198
6,2.46222
7,2.95651
8,2.95651
9,2.48491


Row,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Int64
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,3600.0,11,8370.0,18
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,3050.0,10,5070.0,9
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,3.24,5.832,10.4976,6260.0,19,770.0,4
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,25.0,6.25,15.625,39.0625,420.0,1,6990.0,12
5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,2015.0,6,9470.0,22
6,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.01,0.001,0.0001,1650.0,5,7460.0,14
7,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,42.0,17.64,74.088,311.17,5120.0,17,7280.0,14
8,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,37.0,13.69,50.653,187.416,5240.0,17,5680.0,9
9,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,4040.0,13,8590.0,19


# 2. Data wrangling

2.7. Make an array for our Y variable, which will be the logarithm of wage (lwage column)

In [37]:
#Le cambiamos de nombre a la target a df_lwage puesto que ya existe
 Y = df_lwage
first(Y,5)

Row,lwage
,Float64
1,2.26336
2,3.8728
3,2.40313
4,2.63493
5,3.36198


2.8. Make three new arrays for our predictors:

8.1. The basic model will have the columns sex hsg scl clg ad so we ne exp1 occ2 ind2. Make sure to convert occ2 and ind2 to dummies and to drop the first dummy value to prevent multicolinearity. This means:

Creación de la primera dummy occ2 (Ocupación)

In [38]:
# Número de valores a transformar en dummys
println(sort(unique(data.occ2)))

In [39]:
# Se crea un bucle para poder transformar los valores únicos de la variable occ2 en variables con valores de 0 y 1. 
# La denominación 1 aparecerá cuando la variable coincida con el valor para cada categoría. 
data3 = data2
for i in 2:22
    col = Symbol("occ2_$i")
    data3 = transform(data3, :occ2 => ByRow(x -> x == i ? 1 : 0) => col)
end

println(data3)

Creación de la segunda dummy ind2 (codigo de insdustria)

In [40]:
# Número de valores a transformar en dummys
println(sort(unique(data2.ind2)))

In [41]:
# Se crea un bucle para poder transformar los valores únicos de la variable ind2 en variables con valores de 0 y 1. 
# La denominación 1 aparecerá cuando la variable coincida con el valor para cada categoría. 
for i in 2:21
    col = Symbol("ind2_$i")
    data3 = transform(data3, :ind2 => ByRow(x -> x == i ? 1 : 0) => col)
end

println(data3)

In [42]:
# se crea el modelo básico según las especificaciones, se tiene que eliminar las variables que no son requeridas. 

modelo_basico = select!(data3, Not(["shs", "mw", "occ2", "ind2", "exp2", "exp3", "exp4"]))
modelo_basico

Row,sex,hsg,scl,clg,ad,so,we,ne,exp1,occ,ind,occ2_2,occ2_3,occ2_4,occ2_5,occ2_6,occ2_7,occ2_8,occ2_9,occ2_10,occ2_11,occ2_12,occ2_13,occ2_14,occ2_15,occ2_16,occ2_17,occ2_18,occ2_19,occ2_20,occ2_21,occ2_22,ind2_2,ind2_3,ind2_4,ind2_5,ind2_6,ind2_7,ind2_8,ind2_9,ind2_10,ind2_11,ind2_12,ind2_13,ind2_14,ind2_15,ind2_16,ind2_17,ind2_18,ind2_19,ind2_20,ind2_21
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,7.0,3600.0,8370.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,31.0,3050.0,5070.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,6260.0,770.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,25.0,420.0,6990.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
5,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,22.0,2015.0,9470.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1650.0,7460.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
7,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,42.0,5120.0,7280.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
8,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,37.0,5240.0,5680.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
9,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,31.0,4040.0,8590.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


8.2. The flexible model will have the same columns, and will also include polynomials for experience (exp2 exp3 exp4), as well as the interactions between all experience variables and other variables except for sex. This means

In [43]:
m_flexible = modelo_basico

insertcols!(m_flexible, :exp2 => data2.exp2, :exp3 => data2.exp3, :exp4 => data2.exp4)

m_flexible


Row,sex,hsg,scl,clg,ad,so,we,ne,exp1,occ,ind,occ2_2,occ2_3,occ2_4,occ2_5,occ2_6,occ2_7,occ2_8,occ2_9,occ2_10,occ2_11,occ2_12,occ2_13,occ2_14,occ2_15,occ2_16,occ2_17,occ2_18,occ2_19,occ2_20,occ2_21,occ2_22,ind2_2,ind2_3,ind2_4,ind2_5,ind2_6,ind2_7,ind2_8,ind2_9,ind2_10,ind2_11,ind2_12,ind2_13,ind2_14,ind2_15,ind2_16,ind2_17,ind2_18,ind2_19,ind2_20,ind2_21,exp2,exp3,exp4
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,7.0,3600.0,8370.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.49,0.343,0.2401
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,31.0,3050.0,5070.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,9.61,29.791,92.3521
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,6260.0,770.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.24,5.832,10.4976
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,25.0,420.0,6990.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,6.25,15.625,39.0625
5,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,22.0,2015.0,9470.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.84,10.648,23.4256
6,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1650.0,7460.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.01,0.001,0.0001
7,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,42.0,5120.0,7280.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,17.64,74.088,311.17
8,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,37.0,5240.0,5680.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,13.69,50.653,187.416
9,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,31.0,4040.0,8590.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,9.61,29.791,92.3521


In [44]:
evars = [:exp1, :exp2, :exp3, :exp4]

vars = [
    :hsg, :scl, :clg, :ad, :so, :we, :ne,   
    :occ2_2, :occ2_3, :occ2_4, :occ2_5, :occ2_6, :occ2_7, :occ2_8, :occ2_9, 
    :occ2_10, :occ2_11, :occ2_12, :occ2_13, :occ2_14, :occ2_15, :occ2_16, 
    :occ2_17, :occ2_18, :occ2_19, :occ2_20, :occ2_21, :occ2_22,
    :ind2_2, :ind2_3, :ind2_4, :ind2_5, :ind2_6, :ind2_7, :ind2_8, :ind2_9,
    :ind2_10, :ind2_11, :ind2_12, :ind2_13, :ind2_14, :ind2_15, :ind2_16,
    :ind2_17, :ind2_18, :ind2_19, :ind2_20, :ind2_21
]

# Create interactions
for exp in evars
    for var in vars
        # New column name
        interac = Symbol("$(exp)_x_$(var)")
        
        m_flexible[!, interac] = m_flexible[!, exp] .* m_flexible[!, var]
    end
end

m_flexible

Row,sex,hsg,scl,clg,ad,so,we,ne,exp1,occ,ind,occ2_2,occ2_3,occ2_4,occ2_5,occ2_6,occ2_7,occ2_8,occ2_9,occ2_10,occ2_11,occ2_12,occ2_13,occ2_14,occ2_15,occ2_16,occ2_17,occ2_18,occ2_19,occ2_20,occ2_21,occ2_22,ind2_2,ind2_3,ind2_4,ind2_5,ind2_6,ind2_7,ind2_8,ind2_9,ind2_10,ind2_11,ind2_12,ind2_13,ind2_14,ind2_15,ind2_16,ind2_17,ind2_18,ind2_19,ind2_20,ind2_21,exp2,exp3,exp4,exp1_x_hsg,exp1_x_scl,exp1_x_clg,exp1_x_ad,exp1_x_so,exp1_x_we,exp1_x_ne,exp1_x_occ2_2,exp1_x_occ2_3,exp1_x_occ2_4,exp1_x_occ2_5,exp1_x_occ2_6,exp1_x_occ2_7,exp1_x_occ2_8,exp1_x_occ2_9,exp1_x_occ2_10,exp1_x_occ2_11,exp1_x_occ2_12,exp1_x_occ2_13,exp1_x_occ2_14,exp1_x_occ2_15,exp1_x_occ2_16,exp1_x_occ2_17,exp1_x_occ2_18,exp1_x_occ2_19,exp1_x_occ2_20,exp1_x_occ2_21,exp1_x_occ2_22,exp1_x_ind2_2,exp1_x_ind2_3,exp1_x_ind2_4,exp1_x_ind2_5,exp1_x_ind2_6,exp1_x_ind2_7,exp1_x_ind2_8,exp1_x_ind2_9,exp1_x_ind2_10,exp1_x_ind2_11,exp1_x_ind2_12,exp1_x_ind2_13,exp1_x_ind2_14,exp1_x_ind2_15,exp1_x_ind2_16,exp1_x_ind2_17,exp1_x_ind2_18,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,7.0,3600.0,8370.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.49,0.343,0.2401,0.0,0.0,7.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,⋯
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,31.0,3050.0,5070.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,9.61,29.791,92.3521,0.0,0.0,31.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,6260.0,770.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.24,5.832,10.4976,18.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,25.0,420.0,6990.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,6.25,15.625,39.0625,0.0,0.0,0.0,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
5,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,22.0,2015.0,9470.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.84,10.648,23.4256,0.0,0.0,22.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
6,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1650.0,7460.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.01,0.001,0.0001,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,⋯
7,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,42.0,5120.0,7280.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,17.64,74.088,311.17,42.0,0.0,0.0,0.0,0.0,0.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42.0,0

8.3. The extra-flexible model will include all two-way interactions between variables, except for sex. This means

In [45]:
m_flexible2 = m_flexible


# Variables without "sex" 
vars2 = [
    :exp1, :exp2, :exp3, :exp4, :hsg, :scl, :clg, :ad, :so, :we, :ne,
    :occ2_2, :occ2_3, :occ2_4, :occ2_5, :occ2_6, :occ2_7, :occ2_8, 
    :occ2_9, :occ2_10, :occ2_11, :occ2_12, :occ2_13, :occ2_14, :occ2_15, :occ2_16, 
    :occ2_17, :occ2_18, :occ2_19, :occ2_20, :occ2_21, :occ2_22, 
    :ind2_2, :ind2_3, :ind2_4, :ind2_5, :ind2_6, :ind2_7, :ind2_8, :ind2_9, 
    :ind2_10, :ind2_11, :ind2_12, :ind2_13, :ind2_14, :ind2_15, :ind2_16, 
    :ind2_17, :ind2_18, :ind2_19, :ind2_20, :ind2_21
]

# Create two way interactions
for i in 1:length(vars2)
    for j in (i+1):length(vars2)  # begin from `i+1` to avoid duplicates
        var1 = vars2[i]
        var2 = vars2[j]

        # Name of new interactions
        interac2 = Symbol("$(var1)_x_$(var2)")

        # Create interactions
        m_flexible2[!, interac2] = m_flexible2[!, var1] .* m_flexible2[!, var2]
    end
end

m_flexible2


Row,sex,hsg,scl,clg,ad,so,we,ne,exp1,occ,ind,occ2_2,occ2_3,occ2_4,occ2_5,occ2_6,occ2_7,occ2_8,occ2_9,occ2_10,occ2_11,occ2_12,occ2_13,occ2_14,occ2_15,occ2_16,occ2_17,occ2_18,occ2_19,occ2_20,occ2_21,occ2_22,ind2_2,ind2_3,ind2_4,ind2_5,ind2_6,ind2_7,ind2_8,ind2_9,ind2_10,ind2_11,ind2_12,ind2_13,ind2_14,ind2_15,ind2_16,ind2_17,ind2_18,ind2_19,ind2_20,ind2_21,exp2,exp3,exp4,exp1_x_hsg,exp1_x_scl,exp1_x_clg,exp1_x_ad,exp1_x_so,exp1_x_we,exp1_x_ne,exp1_x_occ2_2,exp1_x_occ2_3,exp1_x_occ2_4,exp1_x_occ2_5,exp1_x_occ2_6,exp1_x_occ2_7,exp1_x_occ2_8,exp1_x_occ2_9,exp1_x_occ2_10,exp1_x_occ2_11,exp1_x_occ2_12,exp1_x_occ2_13,exp1_x_occ2_14,exp1_x_occ2_15,exp1_x_occ2_16,exp1_x_occ2_17,exp1_x_occ2_18,exp1_x_occ2_19,exp1_x_occ2_20,exp1_x_occ2_21,exp1_x_occ2_22,exp1_x_ind2_2,exp1_x_ind2_3,exp1_x_ind2_4,exp1_x_ind2_5,exp1_x_ind2_6,exp1_x_ind2_7,exp1_x_ind2_8,exp1_x_ind2_9,exp1_x_ind2_10,exp1_x_ind2_11,exp1_x_ind2_12,exp1_x_ind2_13,exp1_x_ind2_14,exp1_x_ind2_15,exp1_x_ind2_16,exp1_x_ind2_17,exp1_x_ind2_18,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,7.0,3600.0,8370.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.49,0.343,0.2401,0.0,0.0,7.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,⋯
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,31.0,3050.0,5070.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,9.61,29.791,92.3521,0.0,0.0,31.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,6260.0,770.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.24,5.832,10.4976,18.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,25.0,420.0,6990.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,6.25,15.625,39.0625,0.0,0.0,0.0,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
5,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,22.0,2015.0,9470.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.84,10.648,23.4256,0.0,0.0,22.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
6,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1650.0,7460.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.01,0.001,0.0001,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,⋯
7,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,42.0,5120.0,7280.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,17.64,74.088,311.17,42.0,0.0,0.0,0.0,0.0,0.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42.0,0

## 3. Linear Regressions

9. Split each of the dataframes created (basic, flexible and extra-flexible models) into a training sample (80% of the data) and a test sample.

In [46]:
# Combinar el target 'Y' con el DataFrame de características del modelo básico
data_basico = hcat(Y, modelo_basico)  
rename!(data_basico, 1 => :lwage) 

# Combinar el target 'Y' con el DataFrame de características del modelo flexible
data_flexible = hcat(Y, m_flexible)  
rename!(data_flexible, 1 => :lwage) 

# Combinar el target 'Y' con el DataFrame de características del modelo extra flexible
data_extra_flexible = hcat(Y, m_flexible2)  
rename!(data_extra_flexible, 1 => :lwage)  

# Mostrar los DataFrames resultantes
println("DataFrame del modelo básico con el target:")
display(data_basico)

println("DataFrame del modelo flexible con el target:")
display(data_flexible)

println("DataFrame del modelo extra flexible con el target:")
display(data_extra_flexible)

Row,lwage,sex,hsg,scl,clg,ad,so,we,ne,exp1,occ,ind,occ2_2,occ2_3,occ2_4,occ2_5,occ2_6,occ2_7,occ2_8,occ2_9,occ2_10,occ2_11,occ2_12,occ2_13,occ2_14,occ2_15,occ2_16,occ2_17,occ2_18,occ2_19,occ2_20,occ2_21,occ2_22,ind2_2,ind2_3,ind2_4,ind2_5,ind2_6,ind2_7,ind2_8,ind2_9,ind2_10,ind2_11,ind2_12,ind2_13,ind2_14,ind2_15,ind2_16,ind2_17,ind2_18,ind2_19,ind2_20,ind2_21,exp2,exp3,exp4,exp1_x_hsg,exp1_x_scl,exp1_x_clg,exp1_x_ad,exp1_x_so,exp1_x_we,exp1_x_ne,exp1_x_occ2_2,exp1_x_occ2_3,exp1_x_occ2_4,exp1_x_occ2_5,exp1_x_occ2_6,exp1_x_occ2_7,exp1_x_occ2_8,exp1_x_occ2_9,exp1_x_occ2_10,exp1_x_occ2_11,exp1_x_occ2_12,exp1_x_occ2_13,exp1_x_occ2_14,exp1_x_occ2_15,exp1_x_occ2_16,exp1_x_occ2_17,exp1_x_occ2_18,exp1_x_occ2_19,exp1_x_occ2_20,exp1_x_occ2_21,exp1_x_occ2_22,exp1_x_ind2_2,exp1_x_ind2_3,exp1_x_ind2_4,exp1_x_ind2_5,exp1_x_ind2_6,exp1_x_ind2_7,exp1_x_ind2_8,exp1_x_ind2_9,exp1_x_ind2_10,exp1_x_ind2_11,exp1_x_ind2_12,exp1_x_ind2_13,exp1_x_ind2_14,exp1_x_ind2_15,exp1_x_ind2_16,exp1_x_ind2_17,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,2.26336,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,7.0,3600.0,8370.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.49,0.343,0.2401,0.0,0.0,7.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
2,3.8728,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,31.0,3050.0,5070.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,9.61,29.791,92.3521,0.0,0.0,31.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
3,2.40313,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,6260.0,770.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.24,5.832,10.4976,18.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
4,2.63493,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,25.0,420.0,6990.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,6.25,15.625,39.0625,0.0,0.0,0.0,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,⋯
5,3.36198,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,22.0,2015.0,9470.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.84,10.648,23.4256,0.0,0.0,22.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
6,2.46222,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1650.0,7460.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.01,0.001,0.0001,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,⋯
7,2.95651,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,42.0,5120.0,7280.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,17.64,74.088,311.17,42.0,0.0,0.0,0.0,0.0,0.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

Row,lwage,sex,hsg,scl,clg,ad,so,we,ne,exp1,occ,ind,occ2_2,occ2_3,occ2_4,occ2_5,occ2_6,occ2_7,occ2_8,occ2_9,occ2_10,occ2_11,occ2_12,occ2_13,occ2_14,occ2_15,occ2_16,occ2_17,occ2_18,occ2_19,occ2_20,occ2_21,occ2_22,ind2_2,ind2_3,ind2_4,ind2_5,ind2_6,ind2_7,ind2_8,ind2_9,ind2_10,ind2_11,ind2_12,ind2_13,ind2_14,ind2_15,ind2_16,ind2_17,ind2_18,ind2_19,ind2_20,ind2_21,exp2,exp3,exp4,exp1_x_hsg,exp1_x_scl,exp1_x_clg,exp1_x_ad,exp1_x_so,exp1_x_we,exp1_x_ne,exp1_x_occ2_2,exp1_x_occ2_3,exp1_x_occ2_4,exp1_x_occ2_5,exp1_x_occ2_6,exp1_x_occ2_7,exp1_x_occ2_8,exp1_x_occ2_9,exp1_x_occ2_10,exp1_x_occ2_11,exp1_x_occ2_12,exp1_x_occ2_13,exp1_x_occ2_14,exp1_x_occ2_15,exp1_x_occ2_16,exp1_x_occ2_17,exp1_x_occ2_18,exp1_x_occ2_19,exp1_x_occ2_20,exp1_x_occ2_21,exp1_x_occ2_22,exp1_x_ind2_2,exp1_x_ind2_3,exp1_x_ind2_4,exp1_x_ind2_5,exp1_x_ind2_6,exp1_x_ind2_7,exp1_x_ind2_8,exp1_x_ind2_9,exp1_x_ind2_10,exp1_x_ind2_11,exp1_x_ind2_12,exp1_x_ind2_13,exp1_x_ind2_14,exp1_x_ind2_15,exp1_x_ind2_16,exp1_x_ind2_17,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,2.26336,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,7.0,3600.0,8370.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.49,0.343,0.2401,0.0,0.0,7.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
2,3.8728,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,31.0,3050.0,5070.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,9.61,29.791,92.3521,0.0,0.0,31.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
3,2.40313,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,6260.0,770.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.24,5.832,10.4976,18.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
4,2.63493,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,25.0,420.0,6990.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,6.25,15.625,39.0625,0.0,0.0,0.0,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,⋯
5,3.36198,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,22.0,2015.0,9470.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.84,10.648,23.4256,0.0,0.0,22.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
6,2.46222,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1650.0,7460.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.01,0.001,0.0001,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,⋯
7,2.95651,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,42.0,5120.0,7280.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,17.64,74.088,311.17,42.0,0.0,0.0,0.0,0.0,0.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

Row,lwage,sex,hsg,scl,clg,ad,so,we,ne,exp1,occ,ind,occ2_2,occ2_3,occ2_4,occ2_5,occ2_6,occ2_7,occ2_8,occ2_9,occ2_10,occ2_11,occ2_12,occ2_13,occ2_14,occ2_15,occ2_16,occ2_17,occ2_18,occ2_19,occ2_20,occ2_21,occ2_22,ind2_2,ind2_3,ind2_4,ind2_5,ind2_6,ind2_7,ind2_8,ind2_9,ind2_10,ind2_11,ind2_12,ind2_13,ind2_14,ind2_15,ind2_16,ind2_17,ind2_18,ind2_19,ind2_20,ind2_21,exp2,exp3,exp4,exp1_x_hsg,exp1_x_scl,exp1_x_clg,exp1_x_ad,exp1_x_so,exp1_x_we,exp1_x_ne,exp1_x_occ2_2,exp1_x_occ2_3,exp1_x_occ2_4,exp1_x_occ2_5,exp1_x_occ2_6,exp1_x_occ2_7,exp1_x_occ2_8,exp1_x_occ2_9,exp1_x_occ2_10,exp1_x_occ2_11,exp1_x_occ2_12,exp1_x_occ2_13,exp1_x_occ2_14,exp1_x_occ2_15,exp1_x_occ2_16,exp1_x_occ2_17,exp1_x_occ2_18,exp1_x_occ2_19,exp1_x_occ2_20,exp1_x_occ2_21,exp1_x_occ2_22,exp1_x_ind2_2,exp1_x_ind2_3,exp1_x_ind2_4,exp1_x_ind2_5,exp1_x_ind2_6,exp1_x_ind2_7,exp1_x_ind2_8,exp1_x_ind2_9,exp1_x_ind2_10,exp1_x_ind2_11,exp1_x_ind2_12,exp1_x_ind2_13,exp1_x_ind2_14,exp1_x_ind2_15,exp1_x_ind2_16,exp1_x_ind2_17,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,2.26336,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,7.0,3600.0,8370.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.49,0.343,0.2401,0.0,0.0,7.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
2,3.8728,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,31.0,3050.0,5070.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,9.61,29.791,92.3521,0.0,0.0,31.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
3,2.40313,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,6260.0,770.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.24,5.832,10.4976,18.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
4,2.63493,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,25.0,420.0,6990.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,6.25,15.625,39.0625,0.0,0.0,0.0,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,⋯
5,3.36198,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,22.0,2015.0,9470.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.84,10.648,23.4256,0.0,0.0,22.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
6,2.46222,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1650.0,7460.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.01,0.001,0.0001,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,⋯
7,2.95651,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,42.0,5120.0,7280.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,17.64,74.088,311.17,42.0,0.0,0.0,0.0,0.0,0.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [47]:
using DataFrames, Random

# Fijar la semilla para la reproducibilidad de los resultados
Random.seed!(5)

# Definir una función para dividir un DataFrame en conjunto de entrenamiento y prueba
function split_train_test(df, train_ratio=0.8)
    # Mezclar aleatoriamente las filas del DataFrame
    shuffled_df = shuffle(df)
    
    # Calcular el tamaño del conjunto de entrenamiento
    n_train = round(Int, size(df, 1) * train_ratio)
    
    # Dividir el DataFrame en conjunto de entrenamiento y conjunto de prueba
    train_df = shuffled_df[1:n_train, :]
    test_df = shuffled_df[(n_train+1):end, :]
    
    return train_df, test_df
end

# Aplicar la función de división a cada DataFrame
train_basico, test_basico = split_train_test(data_basico)
train_flexible, test_flexible = split_train_test(data_flexible)
train_extra_flexible, test_extra_flexible = split_train_test(data_extra_flexible)

# Mostrar los resultados
println("Modelo Básico - Entrenamiento:")
display(train_basico)

println("Modelo Básico - Prueba:")
display(test_basico)

println("Modelo Flexible - Entrenamiento:")
display(train_flexible)

println("Modelo Flexible - Prueba:")
display(test_flexible)

println("Modelo Extra Flexible - Entrenamiento:")
display(train_extra_flexible)

println("Modelo Extra Flexible - Prueba:")
display(test_extra_flexible)

Row,lwage,sex,hsg,scl,clg,ad,so,we,ne,exp1,occ,ind,occ2_2,occ2_3,occ2_4,occ2_5,occ2_6,occ2_7,occ2_8,occ2_9,occ2_10,occ2_11,occ2_12,occ2_13,occ2_14,occ2_15,occ2_16,occ2_17,occ2_18,occ2_19,occ2_20,occ2_21,occ2_22,ind2_2,ind2_3,ind2_4,ind2_5,ind2_6,ind2_7,ind2_8,ind2_9,ind2_10,ind2_11,ind2_12,ind2_13,ind2_14,ind2_15,ind2_16,ind2_17,ind2_18,ind2_19,ind2_20,ind2_21,exp2,exp3,exp4,exp1_x_hsg,exp1_x_scl,exp1_x_clg,exp1_x_ad,exp1_x_so,exp1_x_we,exp1_x_ne,exp1_x_occ2_2,exp1_x_occ2_3,exp1_x_occ2_4,exp1_x_occ2_5,exp1_x_occ2_6,exp1_x_occ2_7,exp1_x_occ2_8,exp1_x_occ2_9,exp1_x_occ2_10,exp1_x_occ2_11,exp1_x_occ2_12,exp1_x_occ2_13,exp1_x_occ2_14,exp1_x_occ2_15,exp1_x_occ2_16,exp1_x_occ2_17,exp1_x_occ2_18,exp1_x_occ2_19,exp1_x_occ2_20,exp1_x_occ2_21,exp1_x_occ2_22,exp1_x_ind2_2,exp1_x_ind2_3,exp1_x_ind2_4,exp1_x_ind2_5,exp1_x_ind2_6,exp1_x_ind2_7,exp1_x_ind2_8,exp1_x_ind2_9,exp1_x_ind2_10,exp1_x_ind2_11,exp1_x_ind2_12,exp1_x_ind2_13,exp1_x_ind2_14,exp1_x_ind2_15,exp1_x_ind2_16,exp1_x_ind2_17,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,2.66883,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,1550.0,770.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.36,0.216,0.1296,6.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
2,2.59984,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,21.0,5000.0,6990.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,4.41,9.261,19.4481,0.0,0.0,21.0,0.0,0.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,0.0,0.0,0.0,0.0,0.0,⋯
3,3.67435,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,13.0,710.0,7390.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1.69,2.197,2.8561,0.0,0.0,0.0,13.0,13.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,⋯
4,2.66883,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,32.5,3700.0,9470.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10.5625,34.3281,111.566,0.0,32.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
5,2.44569,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.5,9130.0,4690.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.2025,0.091125,0.0410063,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
6,2.80236,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,4.0,3245.0,8180.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.16,0.064,0.0256,0.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
7,3.55435,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,20.5,4760.0,4670.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,4.2025,8.61513,17.661,0.0,20.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

Row,lwage,sex,hsg,scl,clg,ad,so,we,ne,exp1,occ,ind,occ2_2,occ2_3,occ2_4,occ2_5,occ2_6,occ2_7,occ2_8,occ2_9,occ2_10,occ2_11,occ2_12,occ2_13,occ2_14,occ2_15,occ2_16,occ2_17,occ2_18,occ2_19,occ2_20,occ2_21,occ2_22,ind2_2,ind2_3,ind2_4,ind2_5,ind2_6,ind2_7,ind2_8,ind2_9,ind2_10,ind2_11,ind2_12,ind2_13,ind2_14,ind2_15,ind2_16,ind2_17,ind2_18,ind2_19,ind2_20,ind2_21,exp2,exp3,exp4,exp1_x_hsg,exp1_x_scl,exp1_x_clg,exp1_x_ad,exp1_x_so,exp1_x_we,exp1_x_ne,exp1_x_occ2_2,exp1_x_occ2_3,exp1_x_occ2_4,exp1_x_occ2_5,exp1_x_occ2_6,exp1_x_occ2_7,exp1_x_occ2_8,exp1_x_occ2_9,exp1_x_occ2_10,exp1_x_occ2_11,exp1_x_occ2_12,exp1_x_occ2_13,exp1_x_occ2_14,exp1_x_occ2_15,exp1_x_occ2_16,exp1_x_occ2_17,exp1_x_occ2_18,exp1_x_occ2_19,exp1_x_occ2_20,exp1_x_occ2_21,exp1_x_occ2_22,exp1_x_ind2_2,exp1_x_ind2_3,exp1_x_ind2_4,exp1_x_ind2_5,exp1_x_ind2_6,exp1_x_ind2_7,exp1_x_ind2_8,exp1_x_ind2_9,exp1_x_ind2_10,exp1_x_ind2_11,exp1_x_ind2_12,exp1_x_ind2_13,exp1_x_ind2_14,exp1_x_ind2_15,exp1_x_ind2_16,exp1_x_ind2_17,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,2.7052,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,5.0,650.0,8370.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.25,0.125,0.0625,0.0,0.0,5.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
2,3.05182,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,7.0,950.0,6890.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.49,0.343,0.2401,0.0,0.0,7.0,0.0,7.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,⋯
3,3.14713,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,3.0,2330.0,7860.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.09,0.027,0.0081,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,⋯
4,1.99766,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.5,310.0,8680.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.2025,0.091125,0.0410063,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
5,2.7052,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0,8965.0,2490.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8.41,24.389,70.7281,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0,0.0,0.0,0.0,0.0,0.0,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
6,1.48944,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,5.0,9620.0,7590.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.25,0.125,0.0625,0.0,0.0,5.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,⋯
7,2.78948,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,5.0,310.0,8680.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.25,0.125,0.0625,0.0,0.0,5.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

Row,lwage,sex,hsg,scl,clg,ad,so,we,ne,exp1,occ,ind,occ2_2,occ2_3,occ2_4,occ2_5,occ2_6,occ2_7,occ2_8,occ2_9,occ2_10,occ2_11,occ2_12,occ2_13,occ2_14,occ2_15,occ2_16,occ2_17,occ2_18,occ2_19,occ2_20,occ2_21,occ2_22,ind2_2,ind2_3,ind2_4,ind2_5,ind2_6,ind2_7,ind2_8,ind2_9,ind2_10,ind2_11,ind2_12,ind2_13,ind2_14,ind2_15,ind2_16,ind2_17,ind2_18,ind2_19,ind2_20,ind2_21,exp2,exp3,exp4,exp1_x_hsg,exp1_x_scl,exp1_x_clg,exp1_x_ad,exp1_x_so,exp1_x_we,exp1_x_ne,exp1_x_occ2_2,exp1_x_occ2_3,exp1_x_occ2_4,exp1_x_occ2_5,exp1_x_occ2_6,exp1_x_occ2_7,exp1_x_occ2_8,exp1_x_occ2_9,exp1_x_occ2_10,exp1_x_occ2_11,exp1_x_occ2_12,exp1_x_occ2_13,exp1_x_occ2_14,exp1_x_occ2_15,exp1_x_occ2_16,exp1_x_occ2_17,exp1_x_occ2_18,exp1_x_occ2_19,exp1_x_occ2_20,exp1_x_occ2_21,exp1_x_occ2_22,exp1_x_ind2_2,exp1_x_ind2_3,exp1_x_ind2_4,exp1_x_ind2_5,exp1_x_ind2_6,exp1_x_ind2_7,exp1_x_ind2_8,exp1_x_ind2_9,exp1_x_ind2_10,exp1_x_ind2_11,exp1_x_ind2_12,exp1_x_ind2_13,exp1_x_ind2_14,exp1_x_ind2_15,exp1_x_ind2_16,exp1_x_ind2_17,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,3.51613,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,40.5,7010.0,6390.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,16.4025,66.4301,269.042,0.0,40.5,0.0,0.0,40.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
2,4.05512,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,1020.0,7590.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.04,0.008,0.0016,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,⋯
3,3.36198,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,6.0,1360.0,7290.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.36,0.216,0.1296,0.0,0.0,6.0,0.0,0.0,6.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,⋯
4,2.66883,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,34.0,8030.0,2880.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11.56,39.304,133.634,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34.0,0.0,0.0,0.0,0.0,0.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
5,1.82153,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,2040.0,9160.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,16.0,64.0,256.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
6,4.18029,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,12.0,2100.0,7270.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1.44,1.728,2.0736,0.0,0.0,0.0,12.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,⋯
7,2.59187,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,3600.0,8190.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1.44,1.728,2.0736,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,

Row,lwage,sex,hsg,scl,clg,ad,so,we,ne,exp1,occ,ind,occ2_2,occ2_3,occ2_4,occ2_5,occ2_6,occ2_7,occ2_8,occ2_9,occ2_10,occ2_11,occ2_12,occ2_13,occ2_14,occ2_15,occ2_16,occ2_17,occ2_18,occ2_19,occ2_20,occ2_21,occ2_22,ind2_2,ind2_3,ind2_4,ind2_5,ind2_6,ind2_7,ind2_8,ind2_9,ind2_10,ind2_11,ind2_12,ind2_13,ind2_14,ind2_15,ind2_16,ind2_17,ind2_18,ind2_19,ind2_20,ind2_21,exp2,exp3,exp4,exp1_x_hsg,exp1_x_scl,exp1_x_clg,exp1_x_ad,exp1_x_so,exp1_x_we,exp1_x_ne,exp1_x_occ2_2,exp1_x_occ2_3,exp1_x_occ2_4,exp1_x_occ2_5,exp1_x_occ2_6,exp1_x_occ2_7,exp1_x_occ2_8,exp1_x_occ2_9,exp1_x_occ2_10,exp1_x_occ2_11,exp1_x_occ2_12,exp1_x_occ2_13,exp1_x_occ2_14,exp1_x_occ2_15,exp1_x_occ2_16,exp1_x_occ2_17,exp1_x_occ2_18,exp1_x_occ2_19,exp1_x_occ2_20,exp1_x_occ2_21,exp1_x_occ2_22,exp1_x_ind2_2,exp1_x_ind2_3,exp1_x_ind2_4,exp1_x_ind2_5,exp1_x_ind2_6,exp1_x_ind2_7,exp1_x_ind2_8,exp1_x_ind2_9,exp1_x_ind2_10,exp1_x_ind2_11,exp1_x_ind2_12,exp1_x_ind2_13,exp1_x_ind2_14,exp1_x_ind2_15,exp1_x_ind2_16,exp1_x_ind2_17,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,2.31466,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,11.5,3420.0,8180.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1.3225,1.52087,1.74901,0.0,11.5,0.0,0.0,0.0,0.0,11.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
2,3.5717,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,4800.0,6680.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.04,0.008,0.0016,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
3,3.49651,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,3420.0,7480.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.04,0.008,0.0016,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,⋯
4,2.82298,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,9.5,700.0,7390.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.9025,0.857375,0.814506,0.0,9.5,0.0,0.0,9.5,0.0,0.0,9.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.5,0.0,0.0,0.0,⋯
5,2.48651,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,5700.0,770.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.21,1.331,1.4641,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
6,3.46734,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,10.0,3255.0,8190.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1.0,1.0,1.0,0.0,0.0,10.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
7,3.5717,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,10.0,910.0,6890.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1.0,1.0,1.0,0.0,0.0,10.0,0.0,10.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

Row,lwage,sex,hsg,scl,clg,ad,so,we,ne,exp1,occ,ind,occ2_2,occ2_3,occ2_4,occ2_5,occ2_6,occ2_7,occ2_8,occ2_9,occ2_10,occ2_11,occ2_12,occ2_13,occ2_14,occ2_15,occ2_16,occ2_17,occ2_18,occ2_19,occ2_20,occ2_21,occ2_22,ind2_2,ind2_3,ind2_4,ind2_5,ind2_6,ind2_7,ind2_8,ind2_9,ind2_10,ind2_11,ind2_12,ind2_13,ind2_14,ind2_15,ind2_16,ind2_17,ind2_18,ind2_19,ind2_20,ind2_21,exp2,exp3,exp4,exp1_x_hsg,exp1_x_scl,exp1_x_clg,exp1_x_ad,exp1_x_so,exp1_x_we,exp1_x_ne,exp1_x_occ2_2,exp1_x_occ2_3,exp1_x_occ2_4,exp1_x_occ2_5,exp1_x_occ2_6,exp1_x_occ2_7,exp1_x_occ2_8,exp1_x_occ2_9,exp1_x_occ2_10,exp1_x_occ2_11,exp1_x_occ2_12,exp1_x_occ2_13,exp1_x_occ2_14,exp1_x_occ2_15,exp1_x_occ2_16,exp1_x_occ2_17,exp1_x_occ2_18,exp1_x_occ2_19,exp1_x_occ2_20,exp1_x_occ2_21,exp1_x_occ2_22,exp1_x_ind2_2,exp1_x_ind2_3,exp1_x_ind2_4,exp1_x_ind2_5,exp1_x_ind2_6,exp1_x_ind2_7,exp1_x_ind2_8,exp1_x_ind2_9,exp1_x_ind2_10,exp1_x_ind2_11,exp1_x_ind2_12,exp1_x_ind2_13,exp1_x_ind2_14,exp1_x_ind2_15,exp1_x_ind2_16,exp1_x_ind2_17,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,3.11778,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,5.5,6200.0,770.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.3025,0.166375,0.0915062,0.0,5.5,0.0,0.0,5.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.5,0.0,0.0,0.0,0.0,0.0,5.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
2,2.87855,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,1105.0,6672.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.04,0.008,0.0016,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
3,1.96611,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6.5,4230.0,8660.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.4225,0.274625,0.178506,0.0,6.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
4,3.29298,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,8.5,140.0,2780.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.7225,0.614125,0.522006,0.0,8.5,0.0,0.0,8.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
5,3.36936,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,4.0,2000.0,7870.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.16,0.064,0.0256,0.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,⋯
6,3.6072,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,14.0,9510.0,770.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.96,2.744,3.8416,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
7,3.8526,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,16.0,430.0,9370.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.56,4.096,6.5536,0.0,0.0,0.0,16.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

Row,lwage,sex,hsg,scl,clg,ad,so,we,ne,exp1,occ,ind,occ2_2,occ2_3,occ2_4,occ2_5,occ2_6,occ2_7,occ2_8,occ2_9,occ2_10,occ2_11,occ2_12,occ2_13,occ2_14,occ2_15,occ2_16,occ2_17,occ2_18,occ2_19,occ2_20,occ2_21,occ2_22,ind2_2,ind2_3,ind2_4,ind2_5,ind2_6,ind2_7,ind2_8,ind2_9,ind2_10,ind2_11,ind2_12,ind2_13,ind2_14,ind2_15,ind2_16,ind2_17,ind2_18,ind2_19,ind2_20,ind2_21,exp2,exp3,exp4,exp1_x_hsg,exp1_x_scl,exp1_x_clg,exp1_x_ad,exp1_x_so,exp1_x_we,exp1_x_ne,exp1_x_occ2_2,exp1_x_occ2_3,exp1_x_occ2_4,exp1_x_occ2_5,exp1_x_occ2_6,exp1_x_occ2_7,exp1_x_occ2_8,exp1_x_occ2_9,exp1_x_occ2_10,exp1_x_occ2_11,exp1_x_occ2_12,exp1_x_occ2_13,exp1_x_occ2_14,exp1_x_occ2_15,exp1_x_occ2_16,exp1_x_occ2_17,exp1_x_occ2_18,exp1_x_occ2_19,exp1_x_occ2_20,exp1_x_occ2_21,exp1_x_occ2_22,exp1_x_ind2_2,exp1_x_ind2_3,exp1_x_ind2_4,exp1_x_ind2_5,exp1_x_ind2_6,exp1_x_ind2_7,exp1_x_ind2_8,exp1_x_ind2_9,exp1_x_ind2_10,exp1_x_ind2_11,exp1_x_ind2_12,exp1_x_ind2_13,exp1_x_ind2_14,exp1_x_ind2_15,exp1_x_ind2_16,exp1_x_ind2_17,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,2.59984,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,24.0,5330.0,6890.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,5.76,13.824,33.1776,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,⋯
2,3.36198,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,7700.0,2370.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.36,0.216,0.1296,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
3,3.95981,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,850.0,2070.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.04,0.008,0.0016,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
4,4.05512,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,1020.0,4170.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.04,0.008,0.0016,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
5,2.82298,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,9.0,8140.0,770.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.81,0.729,0.6561,0.0,0.0,9.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
6,3.1797,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,37.0,4710.0,4490.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,13.69,50.653,187.416,37.0,0.0,0.0,0.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
7,2.26336,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,7.0,5620.0,4970.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.49,0.343,0.2401,7.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

10. Estimate all three models

In [48]:
# Se define una función para crear la fórmula dinámicamente usando todas las características excepto la target para crear un modelo que considere todas las otras variables
function create_formula(data::DataFrame, target::Symbol)
    # Ensure the target variable is not included in the features
    data_features = select(data, Not(target))  # Remove the target column from the DataFrame
    
    # Get all column names except the target
    features = names(data_features)  # Now, features will only contain independent variables
    
    # Convert the feature names to symbols
    features_symbols = Symbol.(features)
    
    # Create the formula using the terms as symbols
    return Term(target) ~ sum(Term.(features_symbols))
end

create_formula (generic function with 1 method)

In [49]:
# Mostrar un resumen de los resultados para cada modelo
function display_summary(model, model_name)
    println("Summary of OLS results for ", model_name)
    println(coeftable(model))  # Muestra la tabla de coeficientes
    println("R-squared: ", r2(model))  # Muestra el valor de R-cuadrado
    println("\n")
end

display_summary (generic function with 1 method)

basic

In [50]:
#fórmula del modelo 
formula_basico = create_formula(data_basico, :lwage)
ols_basico = lm(formula_basico, data_basico)
display_summary(ols_basico, "Basic Model")

flexible

In [51]:
formula_flexible = create_formula(data_flexible, :lwage)
ols_flexible = lm(formula_flexible, data_flexible)
display_summary(ols_flexible, "Flexible Model")

extra-flexible model

In [52]:
formula_extra_flexible = create_formula(data_extra_flexible, :lwage)
ols_extra_flexible = lm(formula_extra_flexible, data_extra_flexible)
display_summary(ols_extra_flexible, "Extra Flexible Model")

11. Report the MSE and the R2 for both samples of each model, and the adjusted R2 for the training sample. Do we see overfitting? Which model has the best out-of-sample performance? For reference, you should report the following:

In [53]:
# Function to calculate Mean Squared Error (MSE)
function compute_mse(model, data)
    # Predict the values using the model
    predictions = predict(model, data)
    # Calculate the residuals
    residuals = data.lwage .- predictions
    # Compute MSE
    mse = mean(residuals .^ 2)
    return mse
end

# Function to compute Adjusted R-squared
function adjusted_r2(model, data)
    r2_val = r2(model)  # Compute R-squared
    n = nrow(data)  # Number of observations
    p = length(coef(model)) - 1  # Number of predictors
    adj_r2 = 1 - (1 - r2_val) * (n - 1) / (n - p - 1)  # Adjusted R-squared formula
    return adj_r2
end

# Compute metrics for each model
function report_metrics(formula, train_data, test_data, model_name)
    # Fit the model using the training data
    model_train = lm(formula, train_data)

    # Compute MSE for training and test samples
    mse_train = compute_mse(model_train, train_data)
    mse_test = compute_mse(model_train, test_data)

    # Compute R-squared for training and test samples
    r2_train = r2(model_train)
    r2_test = r2(lm(formula, test_data))

    # Compute Adjusted R-squared for training sample only
    adj_r2_train = adjusted_r2(model_train, train_data)

    # Print the results
    println("Metrics for ", model_name, ":")
    println("MSE (Training Sample): ", mse_train)
    println("MSE (Test Sample): ", mse_test)
    println("R-squared (Training Sample): ", r2_train)
    println("R-squared (Test Sample): ", r2_test)
    println("Adjusted R-squared (Training Sample): ", adj_r2_train)
    println("\n")
end

# Assuming the formulas have been created correctly
# Call the function to report metrics for each model
report_metrics(formula_basico, train_basico, test_basico, "Model 1")
report_metrics(formula_flexible, train_flexible, test_flexible, "Model 2")
report_metrics(formula_extra_flexible, train_extra_flexible, test_extra_flexible, "Model 3")